In [1]:
#JV

In [2]:
import numpy as np

import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler

import os
import gc
import time
import math

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

from tqdm.notebook import tqdm

In [3]:
class LanguageProcessor:

    def __init__(self,language_directory,target_lang_name,mode="train",meta_tokens=True):

        """
        Default Constructor for this class.

        Params:

            language_directory : ex : "aksharantar_sampled/tel/"
            mode : "train" or "test" or "valid", accordingly the appropriate dataset is read.
            meta_tokens : If true creates the first three tokens of the dictionary as <start>,<end>,<pad>.
            
        """

        self.meta_tokens = meta_tokens
        self.language_directory = language_directory
        self.target_lang_name = target_lang
        self.mode = mode
    
        self.source_lang = 0
        self.target_lang = 1

        self.source_max_len = self.find_max_len(self.source_lang)
        self.target_max_len = self.find_max_len(self.target_lang)

        self.max_len = max(self.source_max_len,self.target_max_len)

        self.source_char2id,self.source_id2char = self.build_char_vocab(self.source_lang,self.source_max_len)
        self.target_char2id,self.target_id2char = self.build_char_vocab(self.target_lang,self.target_max_len)


    def find_max_len(self,lang):

        """
        Method to find the maximum length of a word across train/test and validation data.

        This would help in padding, the embedding accordingly.

        Params:

            lang : 0/1 (source/target) language for which the length of the longest word must be found.
        
        """

        train_df = pd.read_csv(self.language_directory+self.target_lang_name+"_train.csv",header=None)
        test_df = pd.read_csv(self.language_directory+self.target_lang_name+"_test.csv",header=None)
        valid_df = pd.read_csv(self.language_directory+self.target_lang_name+"_valid.csv",header=None)

        train_max_len = len(max(list(train_df[lang]), key = len))
        test_max_len = len(max(list(test_df[lang]), key = len))
        valid_max_len = len(max(list(valid_df[lang]), key = len))

        del train_df
        del test_df
        del valid_df

        gc.collect()

        return max(train_max_len,test_max_len,valid_max_len)

    def build_char_vocab(self,lang_id,max_len=None):

        """
        Method to create a vocabulary of characters in language corresponding to lang_id.
        """

        df = pd.read_csv(self.language_directory+self.target_lang_name+"_"+self.mode+".csv",header=None)

        self.data = df.to_numpy()

        lang_chars = []
        lang_words = df[lang_id].to_numpy()
    
        for word in lang_words:
            lang_chars += list(word)
    
        unique_lang_chars =  sorted(list(set(lang_chars)))
    
        start = 0
        
        if self.meta_tokens:
            char2id_dict = {'<start>':0,'<end>':1,'<pad>': 2}
            id2char_dict = {0:'<start>',1:'<end>',2:'<pad>'}
            start = 3
        else:
            char2id_dict = {}
            id2char_dict = {}
    
        for i in range(len(unique_lang_chars)):
            char2id_dict[unique_lang_chars[i]] = i+start
            id2char_dict[i+start] = unique_lang_chars[i]
    
        del df
        del lang_chars
        del unique_lang_chars

        gc.collect()
    
        return char2id_dict,id2char_dict

    def encode_word(self,word,lang_id,padding=False,append_eos = True):

        """
        Method to encode characters of a given word.

        Params:

            word: The word to be encoded.
            lang_id : 0/1 for source/target lang.
            padding : If true, the word encoding would be padded upto max len.
        
        """

        if lang_id == self.source_lang:
            char2id_dict = self.source_char2id
            
        else:
            char2id_dict = self.target_char2id
        
        max_len = self.max_len
        
        word_encoding = []
        
        for i in word:
            word_encoding.append(char2id_dict[i])

        if padding:
            word_encoding += [char2id_dict['<pad>']] * (max_len - len(word_encoding))

        if append_eos:
            word_encoding.append(char2id_dict['<end>'])
        
        return np.array(word_encoding)

    def decode_word(self,code_word,lang_id):

        """
        Method to decode an encoded word.

        Params:

            code_word : The encoded word.
            lang_id : 0/1 for source/target lang.
        """
    
        word = []

        if lang_id == self.source_lang:
            id2char_dict = self.source_id2char
            char2id_dict = self.source_char2id
            
        else:
            id2char_dict = self.target_id2char
            char2id_dict = self.target_char2id
        
        for i in code_word:
            ## if we reached padding, then stop decoding
            if self.meta_tokens and i == char2id_dict['<pad>']:
                break
            
            word.append(id2char_dict[i])
            
        return np.array(word)
            

In [4]:
class WordDataset(Dataset):
    
    def __init__(self, language_processor,device='cpu'):

        self.lp = language_processor
        self.data = self.lp.data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_word, output_word = self.data[idx]
        
        input_sequence = self.lp.encode_word(input_word,self.lp.source_lang,padding=True,append_eos=True)
        output_sequence = self.lp.encode_word(output_word,self.lp.target_lang,padding=True,append_eos=True)
        
        return torch.tensor(input_sequence).to(device), torch.tensor(output_sequence).to(device)


In [5]:
# Step 6: Data Loading
batch_size = 32

base_dir = "aksharantar_sampled/"
target_lang = "tel"

use_meta_tokens = True

lang_dir = base_dir + target_lang + "/"

In [6]:
device = torch.device("mps")

##creating train loader
train_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="train",meta_tokens=use_meta_tokens)
train_dataset = WordDataset(train_lp,device=device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## creating test loader
test_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="test",meta_tokens=use_meta_tokens)
test_dataset = WordDataset(test_lp,device=device)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## creating validation loader
valid_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="valid",meta_tokens=use_meta_tokens)
valid_dataset = WordDataset(valid_lp,device=device)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [7]:
##in principle these are all fixed across train/test/valid data

start_token_id = train_lp.source_char2id['<start>']
end_token_id = train_lp.source_char2id['<end>']
pad_token_id = train_lp.source_char2id['<pad>']

source_max_len = train_lp.source_max_len
target_max_len = train_lp.target_max_len